## Environment setting

In [1]:
import arcpy
import os
from arcpy.sa import *

arcpy.ImportToolbox(r"C:\Users\jyj55\OneDrive - purdue.edu\DeWoody_Lab\Lab work\apps\SDMtoolboxPRO\SDMtoolboxPROv0_9_1.tbx", "sdmtoolbox")
project_folder = r"D:\Research_Data_Backup\Eleutherodactylus_invasion"
gdb = r"D:\Research_Data_Backup\Eleutherodactylus_invasion\Eleutherodactylus_invasion_arcgis\Eleutherodactylus_invasion.gdb"
arcpy.env.workspace = r"D:\Research_Data_Backup\Eleutherodactylus_invasion\Eleutherodactylus_invasion_arcgis"

# Data curation

## Add curated occurrence points of Eleutherodactylus antillensis, E. coqui, E. johnstonei, and E. planirostris for 2020-2021

### Consolidate two land polygon shape files ("land" and "minor islands") from Natural Earth

In [2]:
inFeatures = ["ne_10m_land","ne_10m_minor_islands"]
outFeature = "combined_land"
arcpy.analysis.Union(inFeatures,outFeature)

<Result 'C:\\Users\\jyj55\\Documents\\ArcGIS\\Projects\\Eleutherodactylus_invasion\\combined_land.shp'>

## Remove occurrence points outside of land

In [ ]:
# manually add XY Point data from tables
#species_list = ["Ecoqui_GBIF_20202021","Ejohnstonei_GBIF_20202021", "Eplanirostris_GBIF_20202021"]
#species_list = ["coqui_establishedpops","johnstonei_establishedpops", "planirostris_establishedpops"]
species_list = ["antillensis_rev","coqui_rev", "johnstonei_rev", "martinicensis_rev", "planirostris_rev"]

for point_data in species_list:
    inFeature = point_data
    clipFeature = "combined_land"
    outFeature = point_data + "_onLand"
    arcpy.analysis.Clip(inFeature, clipFeature, outFeature)

In [2]:
# manually export processed occurrence points as shape files because some fields raise issues

## Buffer points of each species by 1km, 60km, 100km

In [ ]:
#species_onland = ["Ecoqui_GBIF_20202021_onLand","Ejohnstonei_GBIF_20202021_onLand", "Eplanirostris_GBIF_20202021_onLand"]
#species_onland = ["coqui_establishedpops_onLand","johnstonei_establishedpops_onLand", "planirostris_establishedpops_onLand"]
species_onland = ["antillensis_rev_onLand","coqui_rev_onLand","johnstonei_rev_onLand", "martinicensis_rev_onLand", "planirostris_rev_onLand"]
#distance_list = [20,40,60,80]
distance_list = [1,60,100]

for point_onland in species_onland:
    for distance in distance_list:
        inFeature = point_onland
        outFeature = f"{point_onland}_{distance}km"
        bufferDistance = f"{distance} Kilometers"
        arcpy.analysis.Buffer(inFeature, outFeature, bufferDistance, "", "", "ALL","")

# Below is just for generating a temporary mask to use in R
for point_onland in species_onland:
    inFeature = point_onland
    outFeature = f"{point_onland}_100km"
    bufferDistance = "100 Kilometers"
    arcpy.analysis.Buffer(inFeature, outFeature, bufferDistance, "", "", "ALL","")


## Clip the buffered area again by land

In [ ]:
#species_onland = ["Ecoqui_GBIF_20202021_onLand","Ejohnstonei_GBIF_20202021_onLand", "Eplanirostris_GBIF_20202021_onLand"]
#species_onland = ["coqui_establishedpops_onLand","johnstonei_establishedpops_onLand", "planirostris_establishedpops_onLand"]
species_onland = ["antillensis_rev_onLand","coqui_rev_onLand","johnstonei_rev_onLand", "martinicensis_rev_onLand", "planirostris_rev_onLand"]
#distance_list = [20,40,60,80]
distance_list = [1,60,100]

for point_onland in species_onland:
    for distance in distance_list:
        inFeature = f"{point_onland}_{distance}km"
        clipFeature = "combined_land"
        outFeature = inFeature + "_onLand"
        arcpy.analysis.Clip(inFeature, clipFeature, outFeature)
        
# Below is just for generating a temporary mask to use in R
for point_onland in species_onland:
    inFeature = f"{point_onland}_100km"
    clipFeature = "combined_land"
    outFeature = inFeature + "_onLand"
    arcpy.analysis.Clip(inFeature, clipFeature, outFeature)

## Export cliped, buffered area shape files and import in R to clip WorldClim bioclimatic data -> not done in revision

In [ ]:
# for some reasons, the code is not working -> use manual tool to export 100km- and 60km-buffered ones to "Eleutherodactylus_invasion_arcgis" folder
#species_onland = ["Ecoqui_GBIF_20202021_onLand","Ejohnstonei_GBIF_20202021_onLand", "Eplanirostris_GBIF_20202021_onLand"]
species_onland = ["coqui_establishedpops_onLand","johnstonei_establishedpops_onLand", "planirostris_establishedpops_onLand"]
distance_list = [20,40,60,80]

for point_onland in species_onland:
    for distance in distance_list:
        inFeature = f"{point_onland}_{distance}km_onLand"
        outFolder = project_folder
        arcpy.conversion.FeatureClassToShapefile(inFeature, outFolder)

# Below is just for generating a temporary mask to use in R
for point_onland in species_onland:
    inFeature = f"{point_onland}_100km_onLand"
    outFolder = project_folder
    arcpy.conversion.FeatureClassToShapefile(inFeature, outFolder)
    
# R code is provided in the R script "comtepm_var.R"

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000732: Output Folder: Dataset D:/Research Data Backup/Eleutherodactylus_invasion/Eleutherodactylus_invasion_arcgis does not exist or is not supported
Failed to execute (FeatureClassToShapefile).


## Resample processed clim data to 30s and clip within 80km buffer -> not done in revision 

In [9]:
# use "_100.tif" file generated from R
species_list = ["Ecoqui", "Ejohnstonei", "Eplanirostris"]
clim_list = ["tmin","tmax","prec"]

refRaster = rf"{project_folder}\HadGEM3-GC31-LL\wc2.1_30s_bioc_HadGEM3-GC31-LL_ssp126_2021-2040.tif"
cell_size = arcpy.management.GetRasterProperties(refRaster, "CELLSIZEX").getOutput(0)

for species in species_list:
    for clim in clim_list:
        inRaster = f"{clim}_{species}_100.tif"
        outRaster = f"{clim}_{species}_100_30s.tif"
        arcpy.management.Resample(inRaster, outRaster, cell_size, "BILINEAR")

ExecuteError: ERROR 000622: Failed to execute (Resample). Parameters are not valid.
ERROR 000628: Cannot set input into parameter cell_size.


In [ ]:
species_list = ["Ecoqui", "Ejohnstonei", "Eplanirostris"]
clim_list = ["tmin","tmax","prec"]
species_clip = ["coqui_establishedpops_onLand_80km_onLand","johnstonei_establishedpops_onLand_80km_onLand", "planirostris_establishedpops_onLand_80km_onLand"]

for i in range(len(species_list)):
    for clim in clim_list:
        species = species_list[i]
        inRaster = f"{clim}_{species}_100_30s.tif"
        inMask = species_clip[i]
        Extent = species_clip[i]
        outRaster = ExtractByMask(inRaster, inMask, "INSIDE", Extent)
        outRaster.save(f"{clim}_{species}_80_30s.tif")

## Conduct thinning points

In [ ]:
# see also model builder ("Spatially Rarefy Occurence Data for SDMs")
# before using the tool, export data as shape files and import them again (excluding unreadable fields; e.g., "gbifID"; everything except "species", "decimalLat", "decimalLon")
# used resolution of 1 km and "World: Azimuthal Equidistant" projection
# added a filed name "sp" and input simplified species name (e.g., Ecoqui) for all fields to match with bias file's name
# after rarefying, confirm the csv format is correctly maintained

## Clip WorldClim data using corresponding buffer (after creating "biovar" in R)

In [ ]:
# Generate temporary (saved in-memory) individual raster layers from present climate multi-band rasters
full_bioclim_list = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
species_list = ["antillensis","coqui","johnstonei","martinicensis","planirostris"]
for number in full_bioclim_list:
    for species in species_list:
        #inRaster = rf"{project_folder}/HadGEM3-GC31-LL/{HadGEM}_{scen}_{year}.tif"
        inRaster = rf"{project_folder}/wc21_25m_bio//biovar_{species}.tif"
        #outLayer = f"{scen}_{year}_bio{number}"
        outLayer = rf"{species}_contemp_bio{number}"
        index = number
        arcpy.MakeRasterLayer_management(inRaster, outLayer, "", "", index)
        outRasterPath = f"{project_folder}/{species}_contemp//bio{number}.tif"
        arcpy.CopyRaster_management(outLayer, outRasterPath, "", "", "", "NONE", "NONE", "", "NONE", "NONE", "TIFF", "NONE")
        
# Clip present WorldClim data -> not done in revision
# files need to be in the project directory or under it; no period or space in the file names
full_bioclim_list = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
species_list = ["antillensis","coqui","johnstonei","martinicensis","planirostris"]
species_clip = ["antillensis_rev_onLand_100km_onLand","coqui_rev_onLand_100km_onLand","johnstonei_rev_onLand_100km_onLand","martinicensis_rev_onLand_100km_onLand","planirostris_rev_onLand_100km_onLand"]

for i in range(len(species_list)):
    for number in full_bioclim_list:
        species = species_list[i]
        inRaster = rf"{project_folder}/wc21_25m_bio/wc21_25m_bio_{number}.tif"
        inMask = species_clip[i]
        #Extent = species_clip[i]
        outRaster = ExtractByMask(inRaster, inMask, "INSIDE")
        outPath = rf"{project_folder}\{species}_contemp\bio{number}.tif"
        outRaster.save(outPath)

In [5]:
# Generate temporary (saved in-memory) individual raster layers from future climate multi-band rasters
#year_list = ["2021-2040","2041-2060","2061-2080","2081-2100"]
year_list = ["2041-2060","2081-2100"]
scenario_list = ["ssp126","ssp585"]
#full_bioclim_list = [1,2,3,5,8,12,14,18,19]
full_bioclim_list = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]
GCM = ["ACCESS-CM2","UKESM1-0-LL","MPI-ESM1-2-HR","INM-CM5-0","IPSL-CM6A-LR"]

for scen in scenario_list:
    for year in year_list:
        for number in full_bioclim_list:
            for model in GCM:
                #inRaster = rf"{project_folder}/HadGEM3-GC31-LL/{HadGEM}_{scen}_{year}.tif"
                inRaster = rf"{project_folder}/{model}/wc2.1_2.5m_bioc_{model}_{scen}_{year}.tif"
                #outLayer = f"{scen}_{year}_bio{number}"
                outLayer = rf"{model}_{scen}_{year}_bio{number}"
                index = number
                arcpy.MakeRasterLayer_management(inRaster, outLayer, "", "", index)
# I ran this step in Python window because Python notebook took too long.

In [10]:
# Clip future WorldClim data
#spp = ["Ecoqui","Ejohnstonei","Eplanirostris"]
spp = ["antillensis","coqui","johnstonei","martinicensis","planirostris"]
#year_list = ["2021-2040","2041-2060","2061-2080","2081-2100"]
year_list = ["2041-2060","2081-2100"]
scenario_list = ["ssp126","ssp585"]
#distance_list = [20,40,60,80]
distance_list = [60,100]
GCM = ["ACCESS-CM2","UKESM1-0-LL","MPI-ESM1-2-HR","INM-CM5-0","IPSL-CM6A-LR"]
full_bioclim_list = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19]

for sp in spp:
#    if sp == "coqui":
#        #bioclim_list = [1,2,3,5,14,19]
#        bioclim_list = [1,2,4,12,15,19]
#    elif sp == "johnstonei":
#        #bioclim_list = [1,2,3,12,14,18]
#        bioclim_list = [1,2,3,12,14]
#    elif sp == "planirostris":
#        #bioclim_list = [1,2,3,5,8,12,14,18]
#        bioclim_list = [1,2,3,5,8,12,14,15,18]

    for i in range(len(year_list)):
        for model in GCM:
            year = year_list[i]
            distance = distance_list[i]
            print("clipping " + year + " bioclim data of " + model + " for " + sp + " by " + str(distance) + " km")
            #clipFeature = f"{gdb}/{sp}_establishedpops_onLand_{distance}km_onLand"
            clipFeature = f"{sp}_rev_onLand_{distance}km_onLand.shp"
            
            for scen in scenario_list:
                for number in full_bioclim_list:
                    inRaster = rf"{model}_{scen}_{year}_bio{number}"
                    #outRaster = f"{project_folder}/E{sp}_{year}/{scen}/bio{number}.tif"
                    outRaster = f"{project_folder}/{sp}_{year}/{model}/{scen}/bio{number}.tif"
                    clip_raster = arcpy.ia.Clip(inRaster, aoi = clipFeature)
                    clip_raster.save(outRaster)

clipping 2021-2040 bioclim data for Ecoqui by 20 km
clipping 2041-2060 bioclim data for Ecoqui by 40 km
clipping 2061-2080 bioclim data for Ecoqui by 60 km
clipping 2081-2100 bioclim data for Ecoqui by 80 km
clipping 2021-2040 bioclim data for Ejohnstonei by 20 km
clipping 2041-2060 bioclim data for Ejohnstonei by 40 km
clipping 2061-2080 bioclim data for Ejohnstonei by 60 km
clipping 2081-2100 bioclim data for Ejohnstonei by 80 km
clipping 2021-2040 bioclim data for Eplanirostris by 20 km
clipping 2041-2060 bioclim data for Eplanirostris by 40 km
clipping 2061-2080 bioclim data for Eplanirostris by 60 km
clipping 2081-2100 bioclim data for Eplanirostris by 80 km


## Correct sampling biases

In [26]:
# SDMtools do not run well with codes -> use manual tools and see model builder ("Gaussian Kernel Density of Sampling Localities")
# while using the tool, the input file should have ".shp" extension, which means that you should browse and select directly the shp file
scientific_names = ["Eleutherodactylus coqui","Eleutherodactylus johnstonei","Eleutherodactylus planirostris"]
species_onland = ["Ecoqui_GBIF_20202021_onLand","Ejohnstonei_GBIF_20202021_onLand", "Eplanirostris_GBIF_20202021_onLand"]

for name in scientific_names:
    for species in species_onland:
        sp = species.split("_")[0]
        if name.split()[1] in species:
            occurenceShapefile = f"{sp}_spatially_rarefied_locs.shp"
            outFile = name
            outFolder = f"{project_folder}/{sp}_bias"
            sizeData = f"{arcpy.env.workspace}/tmin_{sp}_80_30s.tif" 
            biasDistance = 0.25
            arcpy.PseudoabsenceSelectionKD2_sdmtoolbox(occurenceShapefile, outFile, outFolder, sizeData, biasDistance)

Traceback (most recent call last):
  File "C:\Users\jyj55\OneDrive - purdue.edu\DeWoody_Lab\Lab work\apps\SDMtoolboxPRO\SDMtoolboxPROv0_9_1.tbx#PseudoabsenceSelectionKD2_sdmtoolbox.py", line 115, in <module>
  File "C:\Program Files\ArcGIS\Pro\Resources\ArcPy\arcpy\conversion.py", line 635, in RasterToASCII
    raise e
  File "C:\Program Files\ArcGIS\Pro\Resources\ArcPy\arcpy\conversion.py", line 632, in RasterToASCII
    retval = convertArcObjectToPythonObject(gp.RasterToASCII_conversion(*gp_fixargs((in_raster, out_ascii_file), True)))
  File "C:\Program Files\ArcGIS\Pro\Resources\ArcPy\arcpy\geoprocessing\_base.py", line 520, in <lambda>
    return lambda *args: val(*gp_fixargs(args, True))
arcgisscripting.ExecuteError: ERROR 010328: Syntax error at or near symbol SPACE.
ERROR 010267: Syntax error in parsing grid expression.
Failed to execute (RasterToASCII).



ExecuteError: Traceback (most recent call last):
  File "C:\Users\jyj55\OneDrive - purdue.edu\DeWoody_Lab\Lab work\apps\SDMtoolboxPRO\SDMtoolboxPROv0_9_1.tbx#PseudoabsenceSelectionKD2_sdmtoolbox.py", line 115, in <module>
  File "C:\Program Files\ArcGIS\Pro\Resources\ArcPy\arcpy\conversion.py", line 635, in RasterToASCII
    raise e
  File "C:\Program Files\ArcGIS\Pro\Resources\ArcPy\arcpy\conversion.py", line 632, in RasterToASCII
    retval = convertArcObjectToPythonObject(gp.RasterToASCII_conversion(*gp_fixargs((in_raster, out_ascii_file), True)))
  File "C:\Program Files\ArcGIS\Pro\Resources\ArcPy\arcpy\geoprocessing\_base.py", line 520, in <lambda>
    return lambda *args: val(*gp_fixargs(args, True))
arcgisscripting.ExecuteError: ERROR 010328: Syntax error at or near symbol SPACE.
ERROR 010267: Syntax error in parsing grid expression.
Failed to execute (RasterToASCII).


Failed to execute (PseudoabsenceSelectionKD2).


## Define projections

In [18]:
# Define projection
sr = arcpy.SpatialReference("WGS 1984")

#spp = ["Ecoqui","Ejohnstonei","Eplanirostris"]
spp = ["antillensis","coqui","johnstonei","martinicensis","planirostris"]
#full_year_list = ["contemp","2021-2040","2041-2060","2061-2080","2081-2100"]
full_year_list = ["contemp","2041-2060","2081-2100"]
scenario_list = ["ssp126","ssp585"]
GCM = ["ACCESS-CM2","UKESM1-0-LL","MPI-ESM1-2-HR","INM-CM5-0","IPSL-CM6A-LR"]

for sp in spp:
    for year in full_year_list:
        if year == "contemp":
            folder = f"{project_folder}/{sp}_{year}/"
            rasterList = arcpy.da.Walk(folder, topdown = False, datatype = "RasterDataset")
            for dirpath, dirnames, filenames in rasterList:
                for file in filenames:
                    raster_path = os.path.join(dirpath, file)
                    print("Define projection of " + raster_path)
                    arcpy.management.DefineProjection(raster_path, sr)
                    desc = arcpy.Describe(raster_path)
                    print("Current projection of " + raster_path + ": " + desc.spatialReference.name)
        else:
            for model in GCM:
                for scen in scenario_list:
                    folder = f"{project_folder}/{sp}_{year}/{model}/{scen}/"
                    rasterList = arcpy.da.Walk(folder, topdown = False, datatype = "RasterDataset")
                    for dirpath, dirnames, filenames in rasterList:
                        for file in filenames:
                            raster_path = os.path.join(dirpath, file)
                            print("Define projection of " + raster_path)
                            arcpy.management.DefineProjection(raster_path, sr)
                            desc = arcpy.Describe(raster_path)
                            print("Current projection of " + raster_path + ": " + desc.spatialReference.name)

Define projection of D:\Research Data Backup\Eleutherodactylus_invasion/Ecoqui_contemp/bio1.tif
Current projection of D:\Research Data Backup\Eleutherodactylus_invasion/Ecoqui_contemp/bio1.tif: GCS_WGS_1984
Define projection of D:\Research Data Backup\Eleutherodactylus_invasion/Ecoqui_contemp/bio14.tif
Current projection of D:\Research Data Backup\Eleutherodactylus_invasion/Ecoqui_contemp/bio14.tif: GCS_WGS_1984
Define projection of D:\Research Data Backup\Eleutherodactylus_invasion/Ecoqui_contemp/bio19.tif
Current projection of D:\Research Data Backup\Eleutherodactylus_invasion/Ecoqui_contemp/bio19.tif: GCS_WGS_1984
Define projection of D:\Research Data Backup\Eleutherodactylus_invasion/Ecoqui_contemp/bio2.tif
Current projection of D:\Research Data Backup\Eleutherodactylus_invasion/Ecoqui_contemp/bio2.tif: GCS_WGS_1984
Define projection of D:\Research Data Backup\Eleutherodactylus_invasion/Ecoqui_contemp/bio3.tif
Current projection of D:\Research Data Backup\Eleutherodactylus_invasion

Define projection of D:\Research Data Backup\Eleutherodactylus_invasion/Ejohnstonei_2041-2060/ssp126/bio1.tif
Current projection of D:\Research Data Backup\Eleutherodactylus_invasion/Ejohnstonei_2041-2060/ssp126/bio1.tif: GCS_WGS_1984
Define projection of D:\Research Data Backup\Eleutherodactylus_invasion/Ejohnstonei_2041-2060/ssp126/bio12.tif
Current projection of D:\Research Data Backup\Eleutherodactylus_invasion/Ejohnstonei_2041-2060/ssp126/bio12.tif: GCS_WGS_1984
Define projection of D:\Research Data Backup\Eleutherodactylus_invasion/Ejohnstonei_2041-2060/ssp126/bio14.tif
Current projection of D:\Research Data Backup\Eleutherodactylus_invasion/Ejohnstonei_2041-2060/ssp126/bio14.tif: GCS_WGS_1984
Define projection of D:\Research Data Backup\Eleutherodactylus_invasion/Ejohnstonei_2041-2060/ssp126/bio18.tif
Current projection of D:\Research Data Backup\Eleutherodactylus_invasion/Ejohnstonei_2041-2060/ssp126/bio18.tif: GCS_WGS_1984
Define projection of D:\Research Data Backup\Eleuthero

Define projection of D:\Research Data Backup\Eleutherodactylus_invasion/Eplanirostris_2041-2060/ssp585/bio12.tif
Current projection of D:\Research Data Backup\Eleutherodactylus_invasion/Eplanirostris_2041-2060/ssp585/bio12.tif: GCS_WGS_1984
Define projection of D:\Research Data Backup\Eleutherodactylus_invasion/Eplanirostris_2041-2060/ssp585/bio14.tif
Current projection of D:\Research Data Backup\Eleutherodactylus_invasion/Eplanirostris_2041-2060/ssp585/bio14.tif: GCS_WGS_1984
Define projection of D:\Research Data Backup\Eleutherodactylus_invasion/Eplanirostris_2041-2060/ssp585/bio18.tif
Current projection of D:\Research Data Backup\Eleutherodactylus_invasion/Eplanirostris_2041-2060/ssp585/bio18.tif: GCS_WGS_1984
Define projection of D:\Research Data Backup\Eleutherodactylus_invasion/Eplanirostris_2041-2060/ssp585/bio2.tif
Current projection of D:\Research Data Backup\Eleutherodactylus_invasion/Eplanirostris_2041-2060/ssp585/bio2.tif: GCS_WGS_1984
Define projection of D:\Research Data 

Generate PC rasters from bioclim variables

In [ ]:
# refer to Mularo's PC information
spp = ["antillensis","coqui","johnstonei","martinicensis","planirostris"]
full_year_list = ["contemp","2041-2060","2081-2100"]
scenario_list = ["ssp126","ssp585"]
GCM = ["ACCESS-CM2","UKESM1-0-LL","MPI-ESM1-2-HR","INM-CM5-0","IPSL-CM6A-LR"]

for sp in spp:
    for year in full_year_list:
        if year == "contemp":
            folder = f"{project_folder}/{sp}_{year}/"
            bio1 = Raster(f"{folder}/bio1.tif")
            bio2 = Raster(f"{folder}/bio2.tif")
            bio3 = Raster(f"{folder}/bio3.tif")
            bio4 = Raster(f"{folder}/bio4.tif")
            bio5 = Raster(f"{folder}/bio5.tif")
            bio6 = Raster(f"{folder}/bio6.tif")
            bio7 = Raster(f"{folder}/bio7.tif")
            bio8 = Raster(f"{folder}/bio8.tif")
            bio9 = Raster(f"{folder}/bio9.tif")
            bio10 = Raster(f"{folder}/bio10.tif")
            bio11 = Raster(f"{folder}/bio11.tif")
            bio12 = Raster(f"{folder}/bio12.tif")
            bio13 = Raster(f"{folder}/bio13.tif")
            bio14 = Raster(f"{folder}/bio14.tif")
            bio15 = Raster(f"{folder}/bio15.tif")
            bio16 = Raster(f"{folder}/bio16.tif")
            bio17 = Raster(f"{folder}/bio17.tif")
            bio18 = Raster(f"{folder}/bio18.tif")
            bio19 = Raster(f"{folder}/bio19.tif")

            if sp == "antillensis":
                PC1 = (0.246428335443668 * bio1 + (-0.188480299113399) * bio2 + (-0.0809609093922281) * bio3 + (-0.212736590091569) * bio4 + 0.109575328814918 * bio5 + 0.315918863483107 * bio6 + (-0.205130841501161) * bio7 + 0.264080193947722 * bio8 + 0.260241602010308 * bio9 + 0.235200023508927 * bio10 +
                       0.265869854630681 * bio11 + (-0.281215246112788) * bio12 + (-0.268438220049561) * bio13 + (-0.167078319040923) * bio14 + (-0.0772263284151659) * bio15 + (-0.282807041769143) * bio16 + (-0.221401428934925) * bio17 + (-0.275535272941385) * bio18 + (-0.211278127877333) * bio19) 
                PC2 = ((-0.309854094594852) * bio1 + (-0.299769415774214) * bio2 + (-0.197421375104521) * bio3 + (-0.00848782290852746) * bio4 + (-0.420927714702752) * bio5 + (-0.00745503990757048) * bio6 + (-0.294232843559931) * bio7 + (-0.262674345079234) * bio8 + (-0.256216539475725) * bio9 + (-0.312479246019673) * bio10 +
                       (-0.280397719183106) * bio11 + (-0.200088790484015) * bio12 + (-0.20113262956909) * bio13 + (-0.0398799332728656) * bio14 + (-0.184723942766932) * bio15 + (-0.189498238674239) * bio16 + (-0.0900409663858068) * bio17 + (-0.159862494729215) * bio18 + (-0.120771458283346) * bio19) 
            if sp == "coqui":
                PC1 = ((-0.379146621745997) * bio1 + (-0.164882204509632) * bio2 + (-0.0787006210420232) * bio3 + (-0.106601109895622) * bio4 + (-0.377918398462191) * bio5 + (-0.308034851349468) * bio6 + (-0.173912910752117) * bio7 + (-0.366351508060761) * bio8 + (-0.285847837494746) * bio9 + (-0.377747945352945) * bio10 +
                       (-0.368494274843627) * bio11 + 0.0113222546703571 * bio12 + 0.058420386155096 * bio13 + (-0.0204199880742702) * bio14 + 0.0354654720490137 * bio15 + 0.0868564748179374 * bio16 + 0.0048520341383001 * bio17 + (-0.105553674812542) * bio18 + 0.158813080770132 * bio19) 
                PC2 = ((-0.00780056766838187) * bio1 + 0.170964774767252 * bio2 + 0.226994520025782 * bio3 + (-0.141806627005503) * bio4 + 0.0186773856035676 * bio5 + (-0.0660358429782832) * bio6 + 0.100559631582927 * bio7 + 0.0579517157062104 * bio8 + (-0.145622242696168) * bio9 + (-0.0334654120433827) * bio10 +
                       0.0146654286815311 * bio11 + 0.420778665703547 * bio12 + 0.15556189750603 * bio13 + 0.382755700960175 * bio14 + (-0.313203348292901) * bio15 + 0.334470570041923 * bio16 + 0.394594928864304 * bio17 + 0.364988491297926 * bio18 + 0.108206879055005 * bio19) 
            if sp == "johnstonei":
                PC1 = (0.331018793611053 * bio1 + (-0.24518928561779) * bio2 + (-0.167427305778208) * bio3 + 0.117592814683324 * bio4 + 0.27021620456996 * bio5 + 0.318355594340548 * bio6 + (-0.203933303126414) * bio7 + 0.33066987834987 * bio8 + 0.291340453682638 * bio9 + 0.333569273157723 * bio10 +
                       0.314592921124487 * bio11 + (-0.199042996061877) * bio12 + (-0.137051713452538) * bio13 + (-0.139801458404831) * bio14 + 0.0154782905802029 * bio15 + (-0.167747599190321) * bio16 + (-0.159521698581018) * bio17 + (-0.0844704504804847) * bio18 + (-0.170666146200385) * bio19) 
                PC2 = ((-0.145278108792763) * bio1 + 0.0213103899851395 * bio2 + (-0.148187189891491) * bio3 + 0.208957214286525 * bio4 + (-0.1791689286898) * bio5 + (-0.189985065578622) * bio6 + 0.102338263413302 * bio7 + (-0.0725275479996111) * bio8 + (-0.243715475168053) * bio9 + (-0.116073294325209) * bio10 +
                       (-0.196451836270769) * bio11 + (-0.403659349535878) * bio12 + (-0.390876959951828) * bio13 + (-0.2119193958684) * bio14 + (-0.125920249820072) * bio15 + (-0.404126462685794) * bio16 + (-0.236460296136496) * bio17 + (-0.0336367548009155) * bio18 + (-0.347232353080091) * bio19) 
            if sp == "martinicensis":
                PC1 = ((-0.366855057687075) * bio1 + (-0.181489942062603) * bio2 + (-0.0816672061750749) * bio3 + (-0.0794581538342281) * bio4 + (-0.368403462306228) * bio5 + (-0.339592550246846) * bio6 + (-0.0865258441062443) * bio7 + (-0.353787524291279) * bio8 + (-0.353370895482072) * bio9 + (-0.367742100439619) * bio10 +
                       (-0.3650677752075) * bio11 + 0.0347496289513163 * bio12 + (-0.0675348196952853) * bio13 + 0.0405093280802747 * bio14 + (-0.117798883938034) * bio15 + (-0.0105118447333719) * bio16 + 0.0490024578901036 * bio17 + 0.0465813464486182 * bio18 + 0.128264320592254 * bio19) 
                PC2 = ((-0.0116180201856076) * bio1 + 0.123857541963923 * bio2 + 0.306283969087774 * bio3 + (-0.0497840414349954) * bio4 + 0.002726286360712 * bio5 + 0.0716225459040157 * bio6 + (-0.149092155723542) * bio7 + (-0.0281389217148861) * bio8 + (-0.00540854687284686) * bio9 + (-0.00343318850079433) * bio10 +
                       (-0.00985152839583532) * bio11 + (-0.421860221377851) * bio12 + (-0.41839006693658) * bio13 + 0.0078743942435104 * bio14 + (-0.308563675021583) * bio15 + (-0.470971078211896) * bio16 + (-0.100050211288147) * bio17 + (-0.422699382158237) * bio18 + 0.033941361336544 * bio19) 
            if sp == "planirostris":
                PC1 = (0.280650641572156 * bio1 + (-0.210879510680406) * bio2 + 0.279424256138826 * bio3 + (-0.368524702621561) * bio4 + (-0.17261394792479) * bio5 + 0.364664679949536 * bio6 + (-0.368531545058988) * bio7 + (-0.0948963095819512) * bio8 + 0.304702593339404 * bio9 + (-0.0914333297705719) * bio10 +
                       0.351975593096285 * bio11 + 0.153367486958436 * bio12 + 0.186332829398807 * bio13 + 0.0892753915812081 * bio14 + 0.0860844877503539 * bio15 + 0.143255218128482 * bio16 + 0.0550649108843389 * bio17 + (-0.0805489750811154) * bio18 + 0.162864788481445 * bio19) 
                PC2 = ((-0.282269388957481) * bio1 + 0.0643724420483519 * bio2 + 0.0981641213853177 * bio3 + 0.0416570868088451 * bio4 + (-0.222589474224695) * bio5 + (-0.145305205941614) * bio6 + 0.0523216042874905 * bio7 + (-0.305259655227752) * bio8 + (-0.0817071731766737) * bio9 + (-0.333884774771639) * bio10 +
                       (-0.159766571904762) * bio11 + 0.190796858399781 * bio12 + 0.0229496247458009 * bio13 + 0.346941162166856 * bio14 + (-0.320440616177961) * bio15 + 0.0279989338636355 * bio16 + 0.411058591657975 * bio17 + (-0.113532550750272) * bio18 + 0.389774672447526 * bio19) 
            PC1.save(f"{folder}/PC1.tif")
            PC2.save(f"{folder}/PC2.tif")
            
        else:
            for model in GCM:
                for scen in scenario_list:
                    folder = f"{project_folder}/{sp}_{year}/{model}/{scen}/"
                    bio1 = Raster(f"{folder}/bio1.tif")
                    bio2 = Raster(f"{folder}/bio2.tif")
                    bio3 = Raster(f"{folder}/bio3.tif")
                    bio4 = Raster(f"{folder}/bio4.tif")
                    bio5 = Raster(f"{folder}/bio5.tif")
                    bio6 = Raster(f"{folder}/bio6.tif")
                    bio7 = Raster(f"{folder}/bio7.tif")
                    bio8 = Raster(f"{folder}/bio8.tif")
                    bio9 = Raster(f"{folder}/bio9.tif")
                    bio10 = Raster(f"{folder}/bio10.tif")
                    bio11 = Raster(f"{folder}/bio11.tif")
                    bio12 = Raster(f"{folder}/bio12.tif")
                    bio13 = Raster(f"{folder}/bio13.tif")
                    bio14 = Raster(f"{folder}/bio14.tif")
                    bio15 = Raster(f"{folder}/bio15.tif")
                    bio16 = Raster(f"{folder}/bio16.tif")
                    bio17 = Raster(f"{folder}/bio17.tif")
                    bio18 = Raster(f"{folder}/bio18.tif")
                    bio19 = Raster(f"{folder}/bio19.tif")

                    if sp == "antillensis":
                        PC1 = (0.246428335443668 * bio1 + (-0.188480299113399) * bio2 + (-0.0809609093922281) * bio3 + (-0.212736590091569) * bio4 + 0.109575328814918 * bio5 + 0.315918863483107 * bio6 + (-0.205130841501161) * bio7 + 0.264080193947722 * bio8 + 0.260241602010308 * bio9 + 0.235200023508927 * bio10 +
                               0.265869854630681 * bio11 + (-0.281215246112788) * bio12 + (-0.268438220049561) * bio13 + (-0.167078319040923) * bio14 + (-0.0772263284151659) * bio15 + (-0.282807041769143) * bio16 + (-0.221401428934925) * bio17 + (-0.275535272941385) * bio18 + (-0.211278127877333) * bio19) 
                        PC2 = ((-0.309854094594852) * bio1 + (-0.299769415774214) * bio2 + (-0.197421375104521) * bio3 + (-0.00848782290852746) * bio4 + (-0.420927714702752) * bio5 + (-0.00745503990757048) * bio6 + (-0.294232843559931) * bio7 + (-0.262674345079234) * bio8 + (-0.256216539475725) * bio9 + (-0.312479246019673) * bio10 +
                               (-0.280397719183106) * bio11 + (-0.200088790484015) * bio12 + (-0.20113262956909) * bio13 + (-0.0398799332728656) * bio14 + (-0.184723942766932) * bio15 + (-0.189498238674239) * bio16 + (-0.0900409663858068) * bio17 + (-0.159862494729215) * bio18 + (-0.120771458283346) * bio19) 
                    if sp == "coqui":
                        PC1 = ((-0.379146621745997) * bio1 + (-0.164882204509632) * bio2 + (-0.0787006210420232) * bio3 + (-0.106601109895622) * bio4 + (-0.377918398462191) * bio5 + (-0.308034851349468) * bio6 + (-0.173912910752117) * bio7 + (-0.366351508060761) * bio8 + (-0.285847837494746) * bio9 + (-0.377747945352945) * bio10 +
                               (-0.368494274843627) * bio11 + 0.0113222546703571 * bio12 + 0.058420386155096 * bio13 + (-0.0204199880742702) * bio14 + 0.0354654720490137 * bio15 + 0.0868564748179374 * bio16 + 0.0048520341383001 * bio17 + (-0.105553674812542) * bio18 + 0.158813080770132 * bio19) 
                        PC2 = ((-0.00780056766838187) * bio1 + 0.170964774767252 * bio2 + 0.226994520025782 * bio3 + (-0.141806627005503) * bio4 + 0.0186773856035676 * bio5 + (-0.0660358429782832) * bio6 + 0.100559631582927 * bio7 + 0.0579517157062104 * bio8 + (-0.145622242696168) * bio9 + (-0.0334654120433827) * bio10 +
                               0.0146654286815311 * bio11 + 0.420778665703547 * bio12 + 0.15556189750603 * bio13 + 0.382755700960175 * bio14 + (-0.313203348292901) * bio15 + 0.334470570041923 * bio16 + 0.394594928864304 * bio17 + 0.364988491297926 * bio18 + 0.108206879055005 * bio19) 
                    if sp == "johnstonei":
                        PC1 = (0.331018793611053 * bio1 + (-0.24518928561779) * bio2 + (-0.167427305778208) * bio3 + 0.117592814683324 * bio4 + 0.27021620456996 * bio5 + 0.318355594340548 * bio6 + (-0.203933303126414) * bio7 + 0.33066987834987 * bio8 + 0.291340453682638 * bio9 + 0.333569273157723 * bio10 +
                               0.314592921124487 * bio11 + (-0.199042996061877) * bio12 + (-0.137051713452538) * bio13 + (-0.139801458404831) * bio14 + 0.0154782905802029 * bio15 + (-0.167747599190321) * bio16 + (-0.159521698581018) * bio17 + (-0.0844704504804847) * bio18 + (-0.170666146200385) * bio19) 
                        PC2 = ((-0.145278108792763) * bio1 + 0.0213103899851395 * bio2 + (-0.148187189891491) * bio3 + 0.208957214286525 * bio4 + (-0.1791689286898) * bio5 + (-0.189985065578622) * bio6 + 0.102338263413302 * bio7 + (-0.0725275479996111) * bio8 + (-0.243715475168053) * bio9 + (-0.116073294325209) * bio10 +
                               (-0.196451836270769) * bio11 + (-0.403659349535878) * bio12 + (-0.390876959951828) * bio13 + (-0.2119193958684) * bio14 + (-0.125920249820072) * bio15 + (-0.404126462685794) * bio16 + (-0.236460296136496) * bio17 + (-0.0336367548009155) * bio18 + (-0.347232353080091) * bio19) 
                    if sp == "martinicensis":
                        PC1 = ((-0.366855057687075) * bio1 + (-0.181489942062603) * bio2 + (-0.0816672061750749) * bio3 + (-0.0794581538342281) * bio4 + (-0.368403462306228) * bio5 + (-0.339592550246846) * bio6 + (-0.0865258441062443) * bio7 + (-0.353787524291279) * bio8 + (-0.353370895482072) * bio9 + (-0.367742100439619) * bio10 +
                               (-0.3650677752075) * bio11 + 0.0347496289513163 * bio12 + (-0.0675348196952853) * bio13 + 0.0405093280802747 * bio14 + (-0.117798883938034) * bio15 + (-0.0105118447333719) * bio16 + 0.0490024578901036 * bio17 + 0.0465813464486182 * bio18 + 0.128264320592254 * bio19) 
                        PC2 = ((-0.0116180201856076) * bio1 + 0.123857541963923 * bio2 + 0.306283969087774 * bio3 + (-0.0497840414349954) * bio4 + 0.002726286360712 * bio5 + 0.0716225459040157 * bio6 + (-0.149092155723542) * bio7 + (-0.0281389217148861) * bio8 + (-0.00540854687284686) * bio9 + (-0.00343318850079433) * bio10 +
                               (-0.00985152839583532) * bio11 + (-0.421860221377851) * bio12 + (-0.41839006693658) * bio13 + 0.0078743942435104 * bio14 + (-0.308563675021583) * bio15 + (-0.470971078211896) * bio16 + (-0.100050211288147) * bio17 + (-0.422699382158237) * bio18 + 0.033941361336544 * bio19) 
                    if sp == "planirostris":
                        PC1 = (0.280650641572156 * bio1 + (-0.210879510680406) * bio2 + 0.279424256138826 * bio3 + (-0.368524702621561) * bio4 + (-0.17261394792479) * bio5 + 0.364664679949536 * bio6 + (-0.368531545058988) * bio7 + (-0.0948963095819512) * bio8 + 0.304702593339404 * bio9 + (-0.0914333297705719) * bio10 +
                               0.351975593096285 * bio11 + 0.153367486958436 * bio12 + 0.186332829398807 * bio13 + 0.0892753915812081 * bio14 + 0.0860844877503539 * bio15 + 0.143255218128482 * bio16 + 0.0550649108843389 * bio17 + (-0.0805489750811154) * bio18 + 0.162864788481445 * bio19) 
                        PC2 = ((-0.282269388957481) * bio1 + 0.0643724420483519 * bio2 + 0.0981641213853177 * bio3 + 0.0416570868088451 * bio4 + (-0.222589474224695) * bio5 + (-0.145305205941614) * bio6 + 0.0523216042874905 * bio7 + (-0.305259655227752) * bio8 + (-0.0817071731766737) * bio9 + (-0.333884774771639) * bio10 +
                               (-0.159766571904762) * bio11 + 0.190796858399781 * bio12 + 0.0229496247458009 * bio13 + 0.346941162166856 * bio14 + (-0.320440616177961) * bio15 + 0.0279989338636355 * bio16 + 0.411058591657975 * bio17 + (-0.113532550750272) * bio18 + 0.389774672447526 * bio19) 
                    PC1.save(f"{folder}/PC1.tif")
                    PC2.save(f"{folder}/PC2.tif")

## Run SDMtools to model current SDM and project to future climates

In [ ]:
# not working -> move to the R package biomod2
# used rarified occurrence points
# cheked "Create Response Curves", "Create Prediction Pictures", "Do Jackknife to Measure Variable Importance"
# "Automatic Settings" for Regularization Multiplier
# "minimum training presence" for Apply Threshold: make binary model
# checked "Extrapolate Without Clamping"
# "Omission error rate, then AUC" for Best Model Criteria
# "5" for Minimum number of occurrence points to model species' distributions
# "15" for Minimum number of occurrence points to spatially jackknife
# "1" for Replicate of each model parameter class
# "3" for Number of spatial groups and checked "sptaially segreated groups"
# "5" for Number of replicates (for non-spatially jackknifed species)
# "crossvalidate" for Replicate Type (for non-spatially jackknifed species)
# "20" for Percentage Random Test Points 